# Set up environment

**Import libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.data import Dataset
from tensorflow.keras import callbacks
import numpy as np
import cv2
import copy
import time

In [2]:
def image_pooling(image, new_width, new_height):
    return cv2.resize(image, (new_height, new_width), interpolation=cv2.INTER_AREA)

def computeRollingVarianceSum(mean, prev_mean, cur, last):
    return  (cur - prev_mean) * (cur - mean) - (last - prev_mean) * (last - mean)

def swapPositions(li, pos1, pos2):
    li[pos1],li[pos2] = li[pos2],li[pos1]
    return li
def push_down(li):
    for i in range(len(li)-1):
        swapPositions(li, i, i+1)
     #list[-1] is untouched
    return li

In [4]:
def getVariances(video_path, frame_width=128, frame_height=128, target_frame_width=28, target_frame_height=28):
    """
    video_path is the file path to the input video
    Uses functions in the github

    Returns:
    (diff_variances, input_variances)
    """

    rotation_frequency = 150  # enter in revolution per second

    # say the input frames are periodic and can be described by a*sin(bx+c)+d
    # its derivative is a*b*cos(bx+c)
    # its second derivative is -a*b*b*sin(bx+c)
    # so f(x)-f'(x-pi/(2*b))/b should be constant so should f(x)+f"(x)/b/b
    # the second is much better because the first involves a phase shift which
    # requires access to data taken pi/(2*b) ago and will thus lower the processing
    # speed or accuracy.
    # the key is to know b, which is equal to frequency*2pi
    frequency_const = rotation_frequency*2*np.pi

    num_frame = 5

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(5)
    total_frames = int(cap.get(7))-num_frame+1
    print('Video loaded with', total_frames,'frames')

    input_frame = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame+1)]
    input_mean = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)
    input_var_sum = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)

    derivative1 = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame)]
    derivative2 = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame-1)]

    diff = [np.zeros((target_frame_width,target_frame_height),dtype=np.float32) for i in range(num_frame-1)]
    diff_mean = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)
    diff_var_sum = np.zeros((target_frame_width,target_frame_height),dtype=np.float32)

    input_vars = np.zeros((int(cap.get(7)),target_frame_width,target_frame_height),dtype=np.float32)
    diff_vars = np.zeros((int(cap.get(7)),target_frame_width,target_frame_height),dtype=np.float32)

    deltaTime = 1.0/fps
    for i in range(int(cap.get(7))):
        ret, frame = cap.read()
        assert ret
        # shallow copy !!!
        prev_input_mean = copy.deepcopy(input_mean)
        prev_diff_mean = copy.deepcopy(diff_mean)

        input_frame = push_down(input_frame)
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame = image_pooling(frame, target_frame_width, target_frame_height)
        del_input_frame = copy.deepcopy(input_frame[-1])
        input_frame[-1]=frame
        input_mean = (input_mean*(num_frame+1) + frame - del_input_frame)/(num_frame+1)
        # 1 2 3 3 4 6
        #  1 2 3 4 5
        #   3 4 5 6
        derivative1 = push_down(derivative1)
        derivative1[-1]=(input_frame[-1]-input_frame[-2])/deltaTime
        derivative2 = push_down(derivative2)
        derivative2[-1]=(derivative1[-1]-derivative1[-2])/deltaTime

        diff = push_down(diff)
        del_diff = copy.deepcopy(diff[-1])
        diff[-1] = input_frame[-2]+derivative2[-1]/(frequency_const**2)
        diff_mean = (diff_mean*(num_frame-1) + diff[-1] - del_diff)/(num_frame-1)


        input_var_sum += computeRollingVarianceSum(input_mean, prev_input_mean, input_frame[-1], del_input_frame)

        diff_var_sum += computeRollingVarianceSum(diff_mean, prev_diff_mean, diff[-1], del_diff)

        input_var_sum2 = np.var(np.asarray(input_frame), axis=0)
        diff_var_sum2 = np.var(np.asarray(diff), axis=0)

        input_vars[i] = input_var_sum/(num_frame+1)
        diff_vars[i] = diff_var_sum/(num_frame-1)
    return diff_vars, input_vars



In [5]:
def variancesToDataset(diff_variance,input_variance,window_location, batch_size=16):
    """
    diff_variance and input_variance are three dimensional numpy arrays, with third dimension being frame number
    Before inputting into this function, stack every frame from all images to diff_variance and input_variance
    window_location is binary for whether or not there is a window at a pixel in a certain frame

    Returns:
    Dataset
    """
    features = np.transpose(np.array((diff_variance.flatten(),input_variance.flatten())))
    data = (features,window_location.flatten())
    ds = Dataset.from_tensor_slices(data)
    return ds.batch(batch_size).prefetch(1)

In [8]:
def get2D(diff_variances, input_variances):
    """
    diff_variance and input_variance are three dimensional numpy arrays, with third dimension being frame number
    Before inputting into this function, stack every frame from all images to diff_variance and input_variance
    
    Returns:
    2d numpy array with [variance, variance] as each row
    """
    return np.transpose(np.array((diff_variances.flatten(),input_variances.flatten())))

Preprocessing

In [9]:
diff1, input1 = getVariances('outdoor_window_Trim.mp4')
diff2, input2 = getVariances('indoor.avi')

Video loaded with 383 frames
Video loaded with 1990 frames


In [10]:
features1 = get2D(diff1,input1)
features2 = get2D(diff2,input2)
features2 = features2[:features1.shape[0]]
features = np.concatenate((features1, features2))
features[10]

array([1.7371960e-03, 1.2534722e+03], dtype=float32)

In [11]:
labels1 = np.ones(features1.shape[0])
labels2 = np.zeros(features2.shape[0])
labels = np.concatenate((labels1,labels2))

303408

In [25]:
model = Sequential([
    tf.keras.layers.Dense(4, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

In [26]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 3         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [27]:
stopper = callbacks.EarlyStopping(monitor='val_loss',patience=25)
checkpoint = callbacks.ModelCheckpoint('checkpoint.h5', save_best_only=True)
model.fit(
    features,
    labels,
    batch_size=16, 
    epochs=100, 
    validation_split=0.2,
    callbacks=[stopper, checkpoint],
    shuffle=True
)

Epoch 1/100
30341/30341 [==============================] - 33s 1ms/step - loss: 0.8036 - accuracy: 0.7876 - val_loss: 0.5706 - val_accuracy: 0.7809
Epoch 2/100
30341/30341 [==============================] - 31s 1ms/step - loss: 0.3956 - accuracy: 0.8107 - val_loss: 0.7304 - val_accuracy: 0.6482
Epoch 3/100
30341/30341 [==============================] - 35s 1ms/step - loss: 0.3843 - accuracy: 0.8174 - val_loss: 0.6398 - val_accuracy: 0.7262
Epoch 4/100
30341/30341 [==============================] - 47s 2ms/step - loss: 0.3728 - accuracy: 0.8255 - val_loss: 0.5566 - val_accuracy: 0.7863
Epoch 5/100
30341/30341 [==============================] - 42s 1ms/step - loss: 0.3627 - accuracy: 0.8303 - val_loss: 0.5788 - val_accuracy: 0.7742
Epoch 6/100
30341/30341 [==============================] - 48s 2ms/step - loss: 0.3602 - accuracy: 0.8307 - val_loss: 0.5578 - val_accuracy: 0.7868
Epoch 7/100
30341/30341 [==============================] - 42s 1ms/step - loss: 0.3596 - accuracy: 0.8312 - val_

In [28]:
model.save('simpleLogistic')

INFO:tensorflow:Assets written to: simpleLogistic\assets


In [29]:
model.predict(features[:10])

array([[0.35217252],
       [0.35217252],
       [0.35217252],
       [0.35217252],
       [0.35217252],
       [0.35217252],
       [0.35217252],
       [0.35217252],
       [0.3204887 ],
       [0.35217252]], dtype=float32)

In [32]:
features[200:210]

array([[3.4049049e-03, 2.4568057e+03],
       [2.8182040e-03, 2.0334723e+03],
       [3.1537088e-03, 2.2755554e+03],
       [3.5602415e-03, 2.5688889e+03],
       [3.3032717e-03, 2.3834722e+03],
       [3.2031783e-03, 2.3112500e+03],
       [2.5456422e-03, 1.8368057e+03],
       [2.1627853e-03, 1.5605557e+03],
       [2.6349563e-03, 1.9012500e+03],
       [2.5456422e-03, 1.8368057e+03]], dtype=float32)